In [11]:
# importing libraries
import pandas as pd # data science essentials
import matplotlib.pyplot as plt # data visualization
from nltk import ngrams # n-grams
import time
import random
import numpy as np

# Timer Start 
start = time.time()

# setting pandas print options
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)


o =   '/Users/takahiroyamada/Desktop/Challenge/search_engine/bulk_export_20220804 (1)/organization_descriptions.csv'
q =   '/Users/takahiroyamada/Desktop/Challenge/search_engine/bulk_export_20220804 (1)/people_descriptions.csv'

# reading the file into Python
strt = pd.read_csv(  o  , usecols=['description'])

print("Time taken: %s seconds" %(time.time() - start))

Time taken: 6.046619892120361 seconds


In [12]:
start = time.time()

strt = strt.sample(n=1000)

# read data limit to resuce the data size 
#strt =  strt[:1000]

# infomation about data frame
print(strt.info(verbose = True))

#counting null data 
print(strt.isnull().sum())

#Timer show the time 
print("Time taken: %s seconds" %(time.time() - start))

# outputting the first five rows of the dataset
strt.head(n=10)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1000 entries, 344087 to 840044
Data columns (total 1 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   description  1000 non-null   object
dtypes: object(1)
memory usage: 15.6+ KB
None
description    0
dtype: int64
Time taken: 0.44101572036743164 seconds


,description
344087,Cognitive Big Data Systems use creative machin...
567470,Citkar is a platform for mobility in the urban...
11289,Proclivity Systems develops innovative solutio...
466465,SupraG Energy is a graphene technology develop...
207900,Dorkly is a blogging platform that covers ever...
544688,Hempstil is a UK based company supplying high-...
780579,Rockwills Group provides solutions and support...
260851,Paagr Technologies offers healthcare organizat...
713783,Berenson Capital is a sector-focused private e...
53955,CallMeMeeting provides audio conferencing from...


In [17]:
from sklearn.feature_extraction import text
from scipy.cluster.vq import kmeans, vq
from nltk.tokenize import word_tokenize
import re
import spacy

start = time.time()


nlp = spacy.load("en_core_web_sm")

# Function to preprocess text
def preprocess(text):
  	# Create Doc object
    doc = nlp(text, disable=['ner', 'parser'])
    # Generate lemmas
    lemmas = [token.lemma_ for token in doc]
    # Remove stopwords and non-alphabetic characters
    a_lemmas = [lemma for lemma in lemmas 
            if lemma.isalpha() and lemma]
    
    return ' '.join(a_lemmas)
  
# Apply preprocess to ted['transcript']
strt['description'] = strt['description'].apply(preprocess)
strt_rdy =  strt['description']

c_stop_words = text.ENGLISH_STOP_WORDS.union(['chief', 'officer', 'president', 
                                              'served', 'executive','www', 'crunchbase', 'com', 
                                              'organization', 'https','founder', 'ceo', 'cto', 'partner', 
                                              'ventures', 'technology', 'company', 'acquired', 'companies', 
                                              'business','attended', 'university', 'founder', 'school', 'board'])                                       
                                             
#strt = strt.dropna(subset=['description'])
#strt_rdy =  strt['description']
#vectorizer = TfidfVectorizer(ngram_range=(1,1), stop_words=my_stop_words)
tfidf_vectorizer = TfidfVectorizer(stop_words = c_stop_words)
tfidf_matrix = tfidf_vectorizer.fit_transform(strt_rdy)

print("Time taken: %s seconds" %(time.time() - start))

Time taken: 4.6550679206848145 seconds


In [18]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

start = time.time()

num_clusters = 5

# Generate cluster centers through the kmeans function
cluster_centers, distortion = kmeans(tfidf_matrix.todense(), num_clusters)

# Generate terms from the tfidf_vectorizer object
terms = tfidf_vectorizer.get_feature_names_out()

for i in range(num_clusters):
    # Sort the terms and print top 3 terms
    center_terms = dict(zip(terms, list(cluster_centers[i])))
    sorted_terms = sorted(center_terms, key=center_terms.get, reverse=True)
    print(sorted_terms[0:5])
    
print("Time taken: %s seconds" %(time.time() - start))

['provide', 'service', 'offer', 'product', 'base']
['build', 'new', 'help', 'product', 'project']
['equipment', 'brand', 'manufacturer', 'product', 'customer']
['marketing', 'digital', 'social', 'design', 'provide']
['service', 'management', 'solution', 'provide', 'datum']
Time taken: 4.693297863006592 seconds
